# Importing Libraries

In [9]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


# 1: Statistical Risk Minimization

In [10]:
def sampleGenerator(m,n):
    A = np.random.binomial(1, 1/m, (m, n))
    return A

In [11]:
def linearModelSampleGenerator(A, Q):
    #y = Ax+w
    [m, n] = A.shape
    x = np.random.normal(np.zeros((Q, n)), np.sqrt(1/2/n))
    w = np.random.normal(np.zeros((Q, m)), np.sqrt(1/2/m))
    y = x @ A.T + w
    return y, x

In [13]:
def signLinearSampleGenerator(A, Q):
    [m, n] = A.shape
    x = np.random.normal(np.zeros(Q, n), np.sqrt(1/2/n))
    w = np.random.normal(np.zeros(Q, m), np.sqrt(1/2/m))
    y = np.sign(x @ A.T + w)
    return y, x

# 2: Empirical Risk Minimization

1. A solution to (7) is ø($x_{q}$) = $y_{q}$




2. Since the equation does not give us a way of predicting the outputs. There are many solutions to the equation making it difficult to use for predictions. This makes it a characteristic and not a viable solution by itself.

# 3: Learning Parameterizations